# Find mismatches

Source files are created from the APPLAUSE database by this query:

```
SELECT * 
FROM applause_dr4.source 
WHERE plate_id = 19019
ORDER BY source_id
```
with another separate call for the source_calib table.

We look for sources that show up in two separate plates. The ones that don't are the candidates we should look in more detail.

The script exploits parallelism to expedite the search.

In [1]:
import os, io
import csv
from math import sqrt

import multiprocessing as mp
from multiprocessing import Pool

import numpy as np

from astropy import units as u
from astropy.io import fits
from astropy.io.fits import Header, Card
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, Longitude, Latitude

from regions import PolygonSkyRegion
from mocpy import MOC

from settings import get_parameters, fname, current_dataset
from library import Worker, Worker2

In [2]:
par = get_parameters(current_dataset)
par

{'nproc': 8,
 'sextractor_flags': 8,
 'model_prediction': 0.8,
 'max_flux_threshold': 0.2,
 'elongation': 1.7,
 'annular_bin': 7,
 'flag_rim': 0,
 'nbrighter': 150000,
 'sampling_step': 8,
 'min_acceptable_flux': 15000,
 'min_fwhm': 7.0,
 'max_fwhm': 15.0,
 'qfit_max': 5.0,
 'cfit_max': 5.0,
 'invert_east': [False, False],
 'invert_north': [False, False],
 'table1': 'sources_9349.csv',
 'table2': 'sources_9350.csv',
 'table1_calib': 'sources_calib_9349.csv',
 'table2_calib': 'sources_calib_9350.csv',
 'table_matched': 'table_match_9349_9350.fits',
 'table_non_matched': 'table_nomatch_9349_9350.fits',
 'table_psf_nonmatched': 'table_psf_nomatch_9349_9350.fits',
 'image1': 'GS00788_x.fits',
 'image2': 'GS00789_x.fits'}

## Read source tables generated by APPLAUSE database

From both **applause_dr4.source** and **applause_dr4.source_calib**. We need both, but we cannot get a joined version directly from APPLAUSE. Maybe they are limiting the output size. In any way, we get both and join them locally here.

In [3]:
table_src_1 = Table.read(fname(par['table1']), format='ascii.csv')
table_src_2 = Table.read(fname(par['table2']), format='ascii.csv')

table_calib_1 = Table.read(fname(par['table1_calib']), format='ascii.csv')
table_calib_2 = Table.read(fname(par['table2_calib']), format='ascii.csv')

In [4]:
table_1 = join(table_calib_1, table_src_1, keys='source_id')
table_2 = join(table_calib_2, table_src_2, keys='source_id')

In [5]:
print(len(table_1), len(table_2))

77285 64847


In [6]:
table_1

source_id,process_id_1,scan_id_1,plate_id_1,archive_id_1,solution_num,annular_bin_1,dist_center_1,dist_edge_1,sextractor_flags_1,model_prediction_1,ra_icrs,dec_icrs,ra_error,dec_error,gal_lon,gal_lat,ecl_lon,ecl_lat,x_sphere,y_sphere,z_sphere,healpix256,healpix1024,nn_dist,zenith_angle,airmass,natmag,natmag_error,bpmag,bpmag_error,rpmag,rpmag_error,natmag_plate,natmag_correction,natmag_residual,phot_range_flags,phot_calib_flags,color_term,cat_natmag,match_radius,gaiaedr3_id,gaiaedr3_gmag,gaiaedr3_bp_rp,gaiaedr3_dist,gaiaedr3_neighbors,timestamp_insert_1,timestamp_update_1,pos,process_id_2,scan_id_2,plate_id_2,archive_id_2,source_num,x_source,y_source,a_source,b_source,theta_source,erra_source,errb_source,errtheta_source,elongation,x_peak,y_peak,flag_usepsf,x_image,y_image,erra_image,errb_image,errtheta_image,x_psf,y_psf,erra_psf,errb_psf,errtheta_psf,mag_auto,magerr_auto,flux_auto,fluxerr_auto,mag_iso,magerr_iso,flux_iso,fluxerr_iso,flux_max,flux_radius,isoarea,sqrt_isoarea,background,sextractor_flags_2,dist_center_2,dist_edge_2,annular_bin_2,flag_rim,flag_negradius,flag_clean,model_prediction_2,timestamp_insert_2,timestamp_update_2
int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,str10,str10,str10,str10,str10,str10,float64,str15,str15,int64,int64,str9,str10,float64,str19,str4,str10,str12,int64,str29,str29,str42,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,str4,str4,str4,str4,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,int64,int64,float64,str29,str29
40349760000001,34976,12247,9349,102,0,9,6144.9824,2.0025742,24,4.511561e-19,1.283203281304111,55.60929317826631,0.082895,0.0709325,116.37909660828963,-6.672209332915576,31.04422289627915,48.774899454502666,0.5646915150587857,0.012649015122399758,0.8252051231282114,47843,765491,55.53899,27.509445,1.1270059,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,0,2022-06-13 20:53:43.087472+00,2022-06-13 20:53:43.087472+00,"(0.0223961222311517 , 0.970565260667569)",34976,12247,9349,102,1,1188.1964,2.0264132118747247,0.9569564,0.75791895,-1.1912408,0.06713647,0.05307596,-0.0014939102,1.2626104,1188,2,0,1188.1964,2.0025742,0.06713647,0.05307596,-0.0014939102,NULL,NULL,NULL,NULL,NULL,11.922912,0.045624383,170151.3,7148.2944,13.207,0.051990643,52143.496,2496.2927,7578.5264,2.9732122,10,3.1622777,-999.23834,24,6144.9824,2.0025742,9,1,0,0,4.511561e-19,2022-06-13 05:05:31.577726+00,2022-06-13 05:05:31.577726+00
40349760000002,34976,12247,9349,102,0,9,4750.6733,2.9224908,16,1.4123793e-06,0.2781624719464162,55.626867930267615,0.082895,0.0709325,115.8211978423924,-6.548541889616644,30.37266170537604,49.13265629121286,0.5645733642315031,0.0027409410911129503,0.8253783396977762,47807,764915,55.619385,28.048435,1.1325829,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,0,2022-06-13 20:53:43.087472+00,2022-06-13 20:53:43.087472+00,"(0.00485485099095132 , 0.970871997955213)",34976,12247,9349,102,2,3433.6758,2.9470151518937886,1.4373329,0.7585709,-4.9013333,0.08091434,0.041232668,-5.4171886,1.8947904,3434,3,0,3433.6758,2.9224908,0.08091434,0.041232668,-5.4171886,NULL,NULL,NULL,NULL,NULL,11.882136,0.050663263,176662.94,8241.548,12.972132,0.05128892,64736.23,3057.3218,4899.365,2.5927784,14,3.7416575,14265.274,16,4750.6733,2.9224908,9,1,0,0,1.4123793e-06,2022-06-13 05:05:31.577726+00,2022-06-13 05:05:31.577726+00
40349760000003,34976,12247,9349,102,0,9,6499.0225,3.0678332,16,4.894048e-16,357.03660848678555,55.62715587737342,0.082895,0.0709325,114.02500791273691,-6.146240655493036,28.14120477717654,50.25788253931577,0.5638209020471554,-0.029187377036906968,0.8253811770548426,226785,3628575,62

## Remove undesirable sources

Use a variety of criteria from SEXTRACTOR to get rid of possible contaminants.

We want to clean the first table only, since only well-defined star images from the first image should be present in our sample. The second table is taken as is; we want to see everything that is near the sources selected from the firsr table. 

In [7]:
table_1copy = table_1.copy()
table_2copy = table_2.copy()

# Extraction flags 
mask = table_1copy['sextractor_flags_1'] <= par['sextractor_flags']
table_1copy = table_1copy[mask]

# Likelihood of being valid star image 
mask = table_1copy['model_prediction_1'] > par['model_prediction']
table_1copy = table_1copy[mask]

# Round objects 
mask = table_1copy['elongation'] < par['elongation']
table_1copy = table_1copy[mask]

# Ring
mask = table_1copy['annular_bin_1'] <= par['annular_bin']
table_1copy = table_1copy[mask]

# Is source at plate rim ?
mask = table_1copy['flag_rim'] == par['flag_rim']
table_1copy = table_1copy[mask]

# flux threshold - set it to fraction of max peak in image
flux_threshold = max(table_1copy['flux_max']) * par['max_flux_threshold']
mask = table_1copy['flux_max'] > flux_threshold
table_1copy = table_1copy[mask]

print(len(table_1copy), len(table_2copy))

24716 64847


## Remove scanner artifacts

When plates were scanned twice or more, usually in two directions perpendicular to each other, all scans are included in a single table. Stars and artifacts *on* the plate will share the same celestial coordinates in both scans, but scanner-induced artifacts won't, because they move in x,y when the plate is rotated 90 deg. Thus we remove all rows that have no duplicates.  

To expedite the search, we break the job evenly among all available performance CPUs.

In [8]:
# number of processors
nproc = par['nproc']

In [9]:
# The matching code lives in file library.py - it has to be kept in a separate 
# file because of restrictions in name space imposed by the parallelization library.

# Here, the parallelized code stores its product, the list of indices that have duplicates.
matched = []

# callback function to collect results from parallel workers
def collect_result(results):
    matched.extend(results)
    
results = []
pool = Pool(nproc)

row_range = int(len(table_1copy) / nproc)

for p in range(nproc):
    # workers are defined over ranges of rows in the table_1copy table
    worker = Worker2("w"+str(p), table_1copy, int(p*row_range), int((p+1)*row_range))

    r = pool.apply_async(worker, callback=collect_result)
    results.append(r)

for r in results:
    r.wait()

pool.close()

Worker  w0  -  0 3089
Worker  w1  -  3089 6178
Worker  w2  -  6178 9267
Worker  w3  -  9267 12356
Worker  w4  -  12356 15445
Worker  w5  -  15445 18534
Worker  w6  -  18534 21623
Worker  w7  -  21623 24712
w2  -  10% .  6500 19628 40349760025205     40349770022885    0.0772221984789212 0.03511338985333623
w1  -  13% .  3500 12360 40349760015793     40349770009983    0.05490198473125929 0.06214533410116019
w0  -  16% .  500 21838 40349760004920     40349770026779    0.20198509841975465 0.005899326313851816
w2  -  26% .  7000 15094 40349760026974     40349770014945    0.33597934561839793 0.1556322338586824
w1  -  29% .  4000 16695 40349760017407     40349770017715    0.32337637637738226 0.013822128414631152
w2  -  42% .  7500 14768 40349760028747     40349770014309    0.19300619173918676 0.17693896917876373
w0  -  32% .  1000 15746 40349760007035     40349770016019    0.1171023372990021 0.03630057289001343
w1  -  45% .  4500 23570 40349760019057     40349770030069    0.06434772453758342 

In [10]:
print(len(matched), " rows have duplicates")

8842  rows have duplicates


In [11]:
# if a small number of duplicates, or none, was detected, ignore them.
if len(matched) > len(table_1copy) / 10:

    # lots of duplicates: keep only the duplicated rows
    table_1copy = table_1copy[matched]

print(len(table_1copy))

8842


## Look for matches between 1st and 2nd tables

In [12]:
# Same code as above (can't be put in a function to avoid duplication).
#
# It generates the list of indices (in the first table) of all objects 
# for which it could find at least one matching entry in the second table.
matched = []

results = []
pool = Pool(nproc)

row_range = int(len(table_1copy) / nproc)

for p in range(nproc):
    # workers are defined over ranges of rows in the table_1copy table
    worker = Worker("w"+str(p), table_1copy, table_2copy, int(p*row_range), int((p+1)*row_range))

    r = pool.apply_async(worker, callback=collect_result)
    results.append(r)

for r in results:
    r.wait()

pool.close()

Worker  w0  -  0 1105
Worker  w1  -  1105 2210
Worker  w2  -  2210 3315
Worker  w3  -  3315 4420
Worker  w4  -  4420 5525
Worker  w5  -  5525 6630
Worker  w6  -  6630 7735
Worker  w7  -  7735 8840
w0  -  0% .  0 14250 40349770031994     40349780014251    0.5189825131310499 1.4983199214583465
w4  -  7% .  4500 16474 40349760019740     40349780016475    0.7172613191642085 0.3506587018961227
w3  -  16% .  3500 13591 40349760016452     40349780013592    0.7142336406786853 0.005558111513437325
w6  -  33% .  7000 6445 40349760007736     40349780006446    0.05602854075732466 0.05210467157894527
w7  -  24% .  8000 9539 40349760011503     40349780009540    0.12205096791149117 0.15112589492787265
w6  -  78% .  7500 8044 40349760009660     40349780008045    0.3054119471244121 0.24564028995825993
w5  -  43% .  6000 2823 40349760003100     40349780002824    0.004378340122457303 0.45125280595073036
w2  -  26% .  2500 28063 40349760034370     40349780028064    0.10499019560938905 0.07187019749892443


In [13]:
print(len(matched), " sources detected in 1st plate with a match in 2nd plate")

8811  sources detected in 1st plate with a match in 2nd plate


## Get the non-matched objects

Remove the matched rows from the first table. Whatever remains, is the table of objects for which a match couldn't be found.

Remove duplicated indices first, and save original table since removal is done in-place.

In [14]:
table_3 = table_1copy.copy()

m = list(dict.fromkeys(matched))
table_3.remove_rows(m)

In [15]:
table_3

source_id,process_id_1,scan_id_1,plate_id_1,archive_id_1,solution_num,annular_bin_1,dist_center_1,dist_edge_1,sextractor_flags_1,model_prediction_1,ra_icrs,dec_icrs,ra_error,dec_error,gal_lon,gal_lat,ecl_lon,ecl_lat,x_sphere,y_sphere,z_sphere,healpix256,healpix1024,nn_dist,zenith_angle,airmass,natmag,natmag_error,bpmag,bpmag_error,rpmag,rpmag_error,natmag_plate,natmag_correction,natmag_residual,phot_range_flags,phot_calib_flags,color_term,cat_natmag,match_radius,gaiaedr3_id,gaiaedr3_gmag,gaiaedr3_bp_rp,gaiaedr3_dist,gaiaedr3_neighbors,timestamp_insert_1,timestamp_update_1,pos,process_id_2,scan_id_2,plate_id_2,archive_id_2,source_num,x_source,y_source,a_source,b_source,theta_source,erra_source,errb_source,errtheta_source,elongation,x_peak,y_peak,flag_usepsf,x_image,y_image,erra_image,errb_image,errtheta_image,x_psf,y_psf,erra_psf,errb_psf,errtheta_psf,mag_auto,magerr_auto,flux_auto,fluxerr_auto,mag_iso,magerr_iso,flux_iso,fluxerr_iso,flux_max,flux_radius,isoarea,sqrt_isoarea,background,sextractor_flags_2,dist_center_2,dist_edge_2,annular_bin_2,flag_rim,flag_negradius,flag_clean,model_prediction_2,timestamp_insert_2,timestamp_update_2
int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,str10,str10,str10,str10,str10,str10,float64,str15,str15,int64,int64,str9,str10,float64,str19,str4,str10,str12,int64,str29,str29,str42,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,str4,str4,str4,str4,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,int64,int64,float64,str29,str29
40349760026874,34976,12247,9349,102,0,7,4886.4976,918.0039,0,1.0,357.0721234912942,56.947912194020695,0.082895,0.0709325,114.36736090113725,-4.870049088248003,29.47725470555583,51.27327365418548,0.54468929448248,-0.02785846738312704,0.8381750880760185,227150,3634402,29.052227,29.379065,1.1470274,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,0,2022-06-13 20:53:43.087472+00,2022-06-13 20:53:43.087472+00,"(6.23208422201087 , 0.993928569922289)",34976,12247,9349,102,26874,10465.996,5224.605843179607,1.3772416,1.1203021,-43.48527,0.050294947,0.04147386,-45.001236,1.2293484,10466,5225,0,10465.996,5225.0063,0.050294947,0.04147386,-45.001236,NULL,NULL,NULL,NULL,NULL,11.910413,0.041372824,172121.34,6557.2246,12.144569,0.029635819,138730.62,3785.8154,11446.082,1.8594296,20,4.472136,29834.908,0,4886.4976,918.0039,7,0,0,1,1.0,2022-06-13 05:05:31.577726+00,2022-06-13 05:05:31.577726+00
40349760027035,34976,12247,9349,102,0,5,3759.1162,2084.759,0,0.9998652,0.959373888591756,56.94086722541479,0.082895,0.0709325,116.44509979781844,-5.330160626611553,32.07419355908816,49.93922400768153,0.5454278349435863,0.009133624426981856,0.8381080203491545,47869,765905,49.470215,27.33424,1.1252261,NULL,NULL,NULL,NULL,NULL,NULL,0.0,NULL,NULL,0,0,NULL,NULL,0.0,NULL,NULL,NULL,NULL,2,2022-06-13 20:53:43.087472+00,2022-06-13 20:53:43.087472+00,"(0.0167442331135874 , 0.993805612024416)",34976,12247,9349,102,27035,2084.759,5239.883457417215,3.0466714,1.801497,68.99151,0.057163805,0.03457194,69.54246,1.6911887,2084,5238,0,2084.759,5240.218,0.057163805,0.03457194,69.54246,NULL,NULL,NULL,NULL,NULL,10.393724,0.024454745,695841.5,15669.085,10.898099,0.017535172,437280.7,7060.582,13615.618,4.389726,74,8.602325,28719.096,0,3759.1162,2084.759,5,0,0,1,0.9998652,2022-06-13 05:05:31.577726+00,2022-06-13 05:05:31.577726+00
40349760027253,34976,12247,9349,102,0,5,3854.4048,2004.1338,0,1.0,357.57481264098226,56.97313005936326,0.082895,0.0709325,114.6405030036551,-4.911135620699383,29.841001974771032,51.11899121322256,0.5445441137169994,-0.02306296835092644,0.838415056941973,227149,3634392,25.919342,29.108717,1.144013,NULL,NULL,NULL,NULL,NULL,N

## Write result to FITS table

In [16]:
table_3.write(fname(par['table_non_matched']), format='fits', overwrite=True)

In [17]:
# table with the matched objects only (selected by index array m)

table_1copy[m].write(fname(par['table_matched']), format='fits', overwrite=True)

In [18]:
table_1copy[m]

source_id,process_id_1,scan_id_1,plate_id_1,archive_id_1,solution_num,annular_bin_1,dist_center_1,dist_edge_1,sextractor_flags_1,model_prediction_1,ra_icrs,dec_icrs,ra_error,dec_error,gal_lon,gal_lat,ecl_lon,ecl_lat,x_sphere,y_sphere,z_sphere,healpix256,healpix1024,nn_dist,zenith_angle,airmass,natmag,natmag_error,bpmag,bpmag_error,rpmag,rpmag_error,natmag_plate,natmag_correction,natmag_residual,phot_range_flags,phot_calib_flags,color_term,cat_natmag,match_radius,gaiaedr3_id,gaiaedr3_gmag,gaiaedr3_bp_rp,gaiaedr3_dist,gaiaedr3_neighbors,timestamp_insert_1,timestamp_update_1,pos,process_id_2,scan_id_2,plate_id_2,archive_id_2,source_num,x_source,y_source,a_source,b_source,theta_source,erra_source,errb_source,errtheta_source,elongation,x_peak,y_peak,flag_usepsf,x_image,y_image,erra_image,errb_image,errtheta_image,x_psf,y_psf,erra_psf,errb_psf,errtheta_psf,mag_auto,magerr_auto,flux_auto,fluxerr_auto,mag_iso,magerr_iso,flux_iso,fluxerr_iso,flux_max,flux_radius,isoarea,sqrt_isoarea,background,sextractor_flags_2,dist_center_2,dist_edge_2,annular_bin_2,flag_rim,flag_negradius,flag_clean,model_prediction_2,timestamp_insert_2,timestamp_update_2
int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,str10,str10,str10,str10,str10,str10,float64,str15,str15,int64,int64,str9,str10,float64,str19,str4,str10,str12,int64,str29,str29,str42,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,str4,str4,str4,str4,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,int64,int64,float64,str29,str29
40349760006165,34976,12247,9349,102,1,3,2772.6619,1409.8445,0,1.0,359.274173240955,55.99241205162614,0.082895,0.0709325,115.33989948382715,-6.076240910023403,30.030238599817253,49.767019109814164,0.5592578140703118,-0.007085094325144683,0.8289635087503553,226780,3628484,15.987495,28.483313,1.1371964,16.282547,0.20062837,16.08899,0.20081584,15.122486,0.20081584,16.330938,-0.048392236,-0.12709045,0,1,1.2002656,16.155457,0.5455044,1994780247625318784,NULL,0.966503,0.3796068,1,2022-06-13 20:53:43.087472+00,2022-06-13 20:53:43.087472+00,"(6.27051724043517 , 0.977251946434229)",34976,12247,9349,102,6165,5686.055,1409.9448417602541,2.157839,2.0332422,7.8375497,0.039839465,0.03755759,13.223371,1.0612799,5686,1410,0,5686.055,1409.8445,0.039839465,0.03755759,13.223371,NULL,NULL,NULL,NULL,NULL,10.658454,0.029913077,545278.7,15019.304,10.968377,0.017120391,409872.75,6461.4937,10828.025,3.3052878,66,8.124039,31168.906,0,2772.6619,1409.8445,3,0,0,1,1.0,2022-06-13 05:05:31.577726+00,2022-06-13 05:05:31.577726+00
40349760006166,34976,12247,9349,102,1,3,2802.7258,1398.0569,0,1.0,359.41602205564885,55.98864176220931,0.082895,0.0709325,115.41713162557791,-6.096522597938242,30.12341823813182,49.71521341076527,0.5593281859555932,-0.005701059278692855,0.8289267026213634,226774,3628399,12.326807,28.40838,1.1363941,14.507667,0.2880821,14.229846,0.28835106,12.8425865,0.28835106,14.678897,-0.17123055,0.25724125,0,1,1.2002656,14.764908,0.5455044,1994733449659817984,NULL,1.38726,0.40068576,1,2022-06-13 20:53:43.087472+00,2022-06-13 20:53:43.087472+00,"(6.27299296929163 , 0.977186142470153)",34976,12247,9349,102,6166,5372.392,1398.1736774074143,3.1850455,2.8721335,-36.62785,0.029456275,0.02596158,-33.477646,1.1089476,5373,1398,0,5372.392,1398.0569,0.029456275,0.02596158,-33.477646,NULL,NULL,NULL,NULL,NULL,9.459351,0.010156815,1645354.9,15388.182,9.686911,0.008050808,1334245.4,9891.117,18444.596,4.4687576,151,12.288206,30850.104,0,2802.7258,1398.0569,3,0,0,1,1.0,2022-06-13 05:05:31.577726+00,2022-06-13 05:05:31.577726+00
40349760006168,34976,12247,9349,102,1,3,2815.4854,1399.9926,0,1.0,359.46581711729056,55.98881439881399,0.082895,0.0709325,115.4445685047418